In [ ]:
import sys
#!{sys.executable} -m pip install openpyxl

In [ ]:
#!{sys.executable} -m pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline

import openpyxl

In [ ]:
dfFao = pd.read_csv("FAOSTAT.csv")
dfCountries = pd.read_csv("iso-country-codes.csv")
dfConsumption = pd.read_csv("oecd_meat_consumption.csv")

dfFao.head(2)

In [ ]:
dfCountries.head(2)

In [ ]:
dfConsumption.head(2)

## Pasos para limpiar dfCountries
1. Seleccionar solo aquellas columnas necesarias.
2. Renombrar las columnas al igual que las columnas en los otros dataset para luego hacer los joins.
3. Revisar qué valores de dfCountries no hacen correspondencia con los códigos o nombres usados en los otros datasets.

### Paso 1 y 2

In [ ]:
dfCountries = dfCountries[['Alpha-3 code', 'English short name lower case']] \
    .rename(columns={'Alpha-3 code': 'LOCATION', 'English short name lower case':'Area'})
dfCountries.head(5)

### Paso 3: emparejado de valores

In [ ]:
def unirDf(df1, df2, columna):
    df1 = df1.merge(df2, on=columna, how='left', indicator=True)
    print('valores que no hacen correspondencia:')
    print(df1[df1['_merge'] != 'both'])
    return df1


dfUnicosEnConsumo = pd.DataFrame(dfConsumption['LOCATION'].unique(), columns = ['LOCATION'])
dfJoinedConsumptionAndCountries = unirDf(dfUnicosEnConsumo, dfCountries, 'LOCATION')

Esto es esperado, pues el dataset de consumo de OEDC usa esos tres valores para poner valores especiales para su propio análisis.

Ahora, quitamos esos valores y revisamos cuáles valores no están en el dataset de producción de FAO.

In [ ]:
dfConsumptionOK = dfJoinedConsumptionAndCountries[dfJoinedConsumptionAndCountries['_merge'] == 'both'][['Area']]

def corroborarValoresEnFAO(df):
    dfUnicosEnFao = pd.DataFrame(dfFao['Area'].unique(), columns = ['Area'])
    _ = unirDf(df, dfUnicosEnFao, 'Area')
    
corroborarValoresEnFAO(dfConsumptionOK)

Correcciones a valores en dfFao

In [ ]:
def corregirValorEnFao(malo, bueno, columma='Area'):
    dfFao.loc[dfFao[columma] == malo, columma] = bueno

corregirValorEnFao('Republic of Korea', 'South Korea')
corregirValorEnFao('United States of America', 'United States')
corregirValorEnFao('Iran (Islamic Republic of)', 'Iran')
corregirValorEnFao('Russian Federation', 'Russia')
corregirValorEnFao('Viet Nam', 'Vietnam')
corregirValorEnFao('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
corregirValorEnFao('Türkiye', 'Turkey')

corroborarValoresEnFAO(dfConsumptionOK)

In [ ]:
dfCountries.head(5)

----

## Pasos para limpiar dfConsumption
Pasos:
1. Unir los datos de Consumo con los países.
2. Quitar los códigos de 'Location' propios del análisis de la OECD.
3. Remover entradas para Turquía.
4. Quitar columnas innecesarias.

### 1. Unir datos con dataframe de países

In [ ]:
dfConsumption = dfConsumption.merge(dfCountries, on='LOCATION', how='left', indicator=True)
dfConsumption.head(5)

In [ ]:
dfConsumption[dfConsumption['_merge'] != 'both']['LOCATION'].unique()

### 2. Remover Location extras

Remover filas cuando el 'Location' es 'WLD', 'OECD' o 'BRICS'.

Esos son valores totales usados por OECD para comparaciones que no son necesarios para nuestro análisis.

In [ ]:
dfConsumption = dfConsumption[dfConsumption['_merge'] == 'both']
dfConsumption.head(3)

### 3. Columnas innecesarias
Descubriendo cuáles columnas son innecesarias:
- Location (se sustituye por Area).
- Flag Codes que son solo nulos.
- Frecuencia: que siempre es A de Anual.
- Indicador: que siempre es MEATCONSUMP.
- Columnas de códigos redundantes como Year Code, Item Code, Element Code, Area Code
- Measure: solo vamos a ocupar por toneladas, no kg/cápita, pues el dataset de FAO solo tiene toneladas

In [ ]:
dfConsumption['Flag Codes'].unique()

In [ ]:
dfConsumption['FREQUENCY'].unique()

In [ ]:
dfConsumption['INDICATOR'].unique()

En FAO solo tenemos tonnes, mientras que en OECD tenemos tonnes y kg/cápita

In [ ]:
dfConsumption['MEASURE'].unique()

In [ ]:
dfFao['Unit'].unique()

In [ ]:
dfConsumption = dfConsumption[dfConsumption['MEASURE']== 'THND_TONNE']
dfConsumption.head(3)

In [ ]:
dfConsumption = dfConsumption[['Area', 'SUBJECT', 'TIME', 'Value']]
dfConsumption.head(3)

----

## Pasos para limpiar dfFao
Este dataset contiene datos de la producción total en toneladas.

Para nuestro análisis solo nos interesa la producción, para poder compararla con el consumo (dfConsumption).

In [ ]:
dfFao.head(5)

In [ ]:
dfFao['Unit'].unique()

In [ ]:
dfFao[dfFao['Value']==None]

In [ ]:
dfFao = dfFao[['Area', 'Item', 'Year', 'Value']]
dfFao.head(3)

In [ ]:
dfFao = dfFao[dfFao['Value'] != None]
dfFao.head(3)

---
---

# Primeros Gráficos

In [ ]:
corregirValorEnFao('Meat, cattle', 'BEEF', 'Item')
corregirValorEnFao('Meat, chicken', 'POULTRY', 'Item')
corregirValorEnFao('Meat, sheep', 'SHEEP', 'Item')
corregirValorEnFao('Meat, pig', 'PIG', 'Item')


In [ ]:
dfFao['Item'].unique()

In [ ]:
dfFao.head(3)

In [ ]:
dfConsumption.head(3)

In [ ]:
new_df = pd.merge(dfConsumption.rename(columns={'Value': 'Consumption'}), 
                  dfFao.rename(columns={'Value': 'Production'}),
                  how='inner',
                  left_on=['Area','SUBJECT', 'TIME'], right_on = ['Area','Item', 'Year'])
new_df = new_df[['Area', 'Item', 'TIME', 'Consumption', 'Production']]

#cambiar escala de consumo pues está en miles de toneladas
new_df['Consumption'] = new_df['Consumption'] * 1000

new_df.head(10)

In [ ]:
new_df.describe()

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col')
ejes = ax.flatten()
tipos = ['BEEF', 'SHEEP', 'PIG', 'POULTRY']

fig.set_size_inches(18.5, 10.5)

for i in range(4):
    new_df.where(new_df['Item']==tipos[i]) \
        .groupby('TIME') \
        .sum() \
        .plot(title=tipos[i], ax=ejes[i])
fig.savefig('porTipo.png', dpi=100)

In [ ]:
axe = new_df.where(new_df['TIME']<2020).groupby('TIME').sum().plot(title='Total')
axe.get_figure().savefig('total.png', dpi=100)

In [ ]:
paises = new_df['Area'].unique()
print(len(paises))
paises

In [ ]:
axe = sns.heatmap(new_df.corr())
axe.get_figure().savefig('correlation.png', dpi=100)

In [ ]:
new_df.corr()

In [ ]:
new_df.head(3)

Se crea la columna a 'futuro' para poder hacer el merge utilizando ese valor

In [ ]:
new_df["time+1"] = (new_df['TIME']+1)
new_df.head(3)

In [ ]:
df_for_time2 = pd.merge(left=new_df, right=new_df, how='left', 
         left_on=['Area','Item', 'time+1'], right_on = ['Area','Item', 'TIME'])

df_for_time2 = df_for_time2[['Area','Item', 'TIME_x', 'Consumption_x', 'Production_x', 'Consumption_y','Production_y']]
df_for_time2 = df_for_time2.rename(columns={'TIME_x': 'Time'})
df_for_time2.head(3)

In [ ]:
df_for_time2.tail(3)

In [ ]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import category_encoders as ce


In [ ]:
encoder = ce.OneHotEncoder()
new_df_enc = encoder.fit_transform(df_for_time2)

In [ ]:
new_df_enc.columns

In [ ]:
#Dividir en dataframe en datos de entrenamiento y prueba
feature_cols = ['Area_1', 'Area_2', 'Area_3', 'Area_4', 'Area_5', 'Area_6', 'Area_7',
       'Area_8', 'Area_9', 'Area_10', 'Area_11', 'Area_12', 'Area_13',
       'Area_14', 'Area_15', 'Area_16', 'Area_17', 'Area_18', 'Area_19',
       'Area_20', 'Area_21', 'Area_22', 'Area_23', 'Area_24', 'Area_25',
       'Area_26', 'Area_27', 'Area_28', 'Area_29', 'Area_30', 'Area_31',
       'Area_32', 'Area_33', 'Area_34', 'Area_35', 'Item_1', 'Item_2',
       'Item_3', 'Item_4', 'Time', 'Consumption_x']

dfTrain = new_df_enc[new_df_enc.Time < 2019 ]
X_train = dfTrain[feature_cols]
Y_train = dfTrain.Consumption_y

dfTest  = new_df_enc[new_df_enc.Time == 2019]
X_test = dfTest[feature_cols]
Y_test = dfTest.Consumption_y

In [ ]:
def printTotales():
    totalRecords = new_df_enc[new_df_enc.Time <= 2019].shape[0]
    print('total de datos iniciales:', new_df_enc.shape[0])
    print('total de datos sin nulos (por el T, T+1):', totalRecords)
    print('datos de entrenamiento:', dfTrain.shape[0], " = {:.2f}%".format(dfTrain.shape[0] / totalRecords * 100))
    print('datos de prueba:', dfTest.shape[0], " = {:.2f}%".format(dfTest.shape[0] / totalRecords * 100))
printTotales()

In [ ]:
# Create Decision Tree classifier object
clf = DecisionTreeRegressor()
# Train Decision Tree Classifier
clf.fit(X_train,Y_train)
#Predict the response for test dataset
Y_pred = clf.predict(X_test)

df_pred_test=pd.DataFrame({'Actual':Y_test, 'Predicted':Y_pred})
df_pred_test

In [ ]:
def neural_model(look_back, trainX, trainY, epochs, batch_size, testX, testY):
    model = Sequential()
    model.add(Dense(32, input_dim=look_back, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(testX,testY), 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
    return model, history

In [ ]:
def history_model_plot(dataset, look_back, trainPredict, testPredict):
    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(dataset)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(dataset)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(trainPredict)+look_back:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    print(dataset.shape)
    print(trainPredict.shape)
    print(testPredict.shape)
    plt.plot(dataset)
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()

In [ ]:
def model_loss(history):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Test Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show();

In [ ]:
def prediction_plot(testY, test_predict, l):
    len_prediction=[x for x in range(len(testY))]
    plt.figure(figsize=(8,4))
    plt.plot(len_prediction, testY[:l], marker='.', label="actual")
    plt.plot(len_prediction, test_predict[:l], 'r', label="prediction")
    plt.tight_layout()
    sns.despine(top=True)
    plt.subplots_adjust(left=0.07)
    plt.ylabel('Ads Daily Spend', size=15)
    plt.xlabel('Time step', size=15)
    plt.legend(fontsize=15)
    plt.show();

In [ ]:
model, history = neural_model(X_train.shape[1], X_train, Y_train, 300, 20, X_test, Y_test)

In [ ]:
trainScore = model.evaluate(X_test, Y_test, verbose=0)
print('Train Score: %.2f MSE (%.2f MAE)' % (trainScore[0], trainScore[1]))

testScore = model.evaluate(X_test, Y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f MAE)' % (trainScore[0], trainScore[1])) 

In [ ]:
model_loss(history)

In [ ]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

In [ ]:
history_model_plot(dataset, look_back, trainPredict, testPredict)

In [ ]:
prediction_plot(Y_test, testPredict, Y_test.shape[0])

In [ ]:
X_test